In [1]:
# import das bibliotecas
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import collections
import os
from os.path import isfile, isdir, join

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///home/jovyan/jdbc/postgresql-42.2.17.jar pyspark-shell'

In [3]:
db_host = os.getenv('POSTGRES_HOST')
db_port = os.getenv('POSTGRES_PORT')
db_name = os.getenv('POSTGRES_DB')
db_user = os.getenv('POSTGRES_USER')
db_pass = os.getenv('POSTGRES_PASSWORD')

db_driver = "org.postgresql.Driver"
db_url = "jdbc:postgresql://"+db_host+":"+db_port+"/" + db_name

In [4]:
# quando for True, as tabelas das dimensões serão recriadas e carregadas
carregar_dimensoes = True

In [5]:
# inicialização do spark
conf = SparkConf() \
        .setMaster("local[*]") \
        .setAppName("ETL-CarregarDadosNasDimensoes") \
        .set("spark.executor.memory", "8g") \
        .set("spark.driver.memory", "8g") \
        .set("spark.driver.maxResultSize", "2g") \
        .set("spark.ui.enabled", "true") \
        .set("spark.sql.shuffle.partitions" , "800") \
        .set("spark.sql.execution.arrow.pyspark.enabled" , "false") \

spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext

In [6]:
# definindo o schema dos dados para leitura dos arquivos JSON
schema = StructType([
    StructField("dadosBasicos", StructType([
        StructField("assunto", ArrayType(
            StructType([
                StructField("assuntoLocal", StructType([
                    StructField("codigoAssunto", LongType(), True),
                    StructField("codigoPaiNacional", LongType(), True),
                    StructField("descricao", StringType(), True)
                ]), True),
                StructField("codigoNacional", LongType(), True),
                StructField("principal", BooleanType(), True)
            ]),
        ), True),
        StructField('classeProcessual', LongType(), True),
        StructField('codigoLocalidade', StringType(), True),
        StructField('competencia', StringType(), True),
        StructField('dataAjuizamento', StringType(), True),
        StructField('dscSistema', StringType(), True),
        StructField('nivelSigilo', LongType(), True),
        StructField('numero', StringType(), True),
        StructField("orgaoJulgador", StructType([
            StructField("codigoMunicipioIBGE", LongType(), True),
            StructField("codigoOrgao", StringType(), True),
            StructField("instancia", StringType(), True),
            StructField("nomeOrgao", StringType(), True)
        ]), True),
        StructField('procEl', LongType(), True),
        StructField("tamanhoProcesso", StringType(), True),
        StructField("totalAssuntos", LongType(), True),
        StructField("valorCausa", StringType(), True)       
    ]), True),
    StructField("grau", StringType(), True),
    StructField("millisInsercao", LongType(), True),
    StructField("movimento", ArrayType(     
        StructType([
            StructField("complementoNacional", ArrayType(
                StructType([
                    StructField("codComplemento", LongType(), True),
                    StructField("codComplementoTabelado", LongType(), True),
                    StructField("descricaoComplemento", StringType(), True),
                ])
            ), True),
            StructField("dataHora", StringType(), True),
            StructField("idDocumentoVinculado", ArrayType(
                StringType(),
            ), True),
            StructField("identificadorMovimento", StringType(), True),
            StructField("movimentoLocal", StructType([
                StructField('codigoMovimento', LongType(), True),
                StructField('codigoPaiNacional', LongType(), True)
            ]), True),
            StructField("movimentoNacional", StructType([
                StructField('codigoNacional', LongType(), True)
            ]), True),
            StructField("nivelSigilo", StringType(), True),
            StructField("orgaoJulgador", StructType([
                StructField("codigoMunicipioIBGE", LongType(), True),
                StructField("codigoOrgao", StringType(), True),
                StructField("instancia", StringType(), True),
                StructField("nomeOrgao", StringType(), True)
            ]), True),
            StructField("tipoDecisao", StringType(), True),
            StructField("tipoResponsavelMovimento", StringType(), True)
        ]),
    ), True),
    StructField("siglaTribunal", StringType(), True)
])

In [7]:
# carrega o CSV de classes e faz a carga da dimensão
df_classes = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_classes.csv")

df_classes.createOrReplaceTempView("classes")
   
df_qry_classes = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "sigla," + 
    "cod_pai AS codpai " +    
    "FROM classes "
)

if carregar_dimensoes :
    df_qry_classes.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.classe") \
        .save()

print("tabela inovacnj.classe criada.")

tabela inovacnj.classe criada.


In [8]:
# carrega o CSV de assuntos e faz a carga da dimensão
df_assuntos = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_assuntos.csv")

df_assuntos.createOrReplaceTempView("assuntos")
   
df_qry_assuntos = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "cod_pai AS codpai " +    
    "FROM assuntos "
)

if carregar_dimensoes :
    df_qry_assuntos.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.assunto") \
        .save()

print("tabela inovacnj.assunto criada.")

tabela inovacnj.assunto criada.


In [9]:
# carrega o CSV de movimentos e faz a carga da dimensão
df_movimentos = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/sgt_movimentos.csv")

# cria uma view temporaria dos movimentos
df_movimentos.createOrReplaceTempView("movimentos")

# carrega o dataframe de movimentos nacionais (com nossa classificacao de fases e natureza)
df_movimentosNac = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/MovimentosNacionais.csv")

df_movimentosNac.createOrReplaceTempView("movimentos_nac")

df_qry_movimentosNac = spark.sql(
    "SELECT " +
    "trim(substring_index(MOVIMENTO, '-', 1)) AS codmovimento, " + 
    "trim(substring_index(MOVIMENTO, '-', -1)) AS descmovimento, " + 
    "CASE WHEN NATUREZA IS NULL THEN 'GERAL' ELSE NATUREZA END AS natureza, " +
    "CASE WHEN FASE IS NULL THEN 'F0 - NÃO CLASSIFICADO' ELSE FASE END AS fase " +
    "FROM movimentos_nac " +
    "WHERE RELEVANCIA = 1"
)

df_movimentos_join = df_movimentos \
    .join(df_qry_movimentosNac, df_movimentos["codigo"] == df_qry_movimentosNac["codmovimento"], "left")

df_movimentos_join.createOrReplaceTempView("movimentos_com_fase")

df_qry_movimentos = spark.sql(
    "SELECT " +
    "codigo AS cod," + 
    "descricao," + 
    "natureza, " +
    "fase, " +
    "cod_pai AS codpai " +    
    "FROM movimentos_com_fase "
)

if carregar_dimensoes :
    df_qry_movimentos.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.movimentocnj") \
        .save()

print("tabela inovacnj.movimentocnj criada.")

tabela inovacnj.movimentocnj criada.


In [10]:
# carrega o CSV de tribunal e faz a carga da dimensão
df_tribunais = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",",") \
    .csv("./base/tribunal.csv")

df_tribunais.createOrReplaceTempView("tribunais")

df_qry_df_tribunais = spark.sql(
    "SELECT * " +
    "FROM tribunais "
)

if carregar_dimensoes :
    df_qry_df_tribunais.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.tribunal") \
        .save()

print("tabela inovacnj.tribunal criada.")

tabela inovacnj.tribunal criada.


In [14]:
# carrega o CSV de serventia e faz a carga da dimensão
df_serventias = spark.read \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .csv("./base/mpm_serventias.csv")

df_serventias.createOrReplaceTempView("serventias")

df_qry_serventias = spark.sql(
    "SELECT " +
    "SEQ_ORGAO AS cod, " + 
    "DSC_ORGAO AS descricao, " + 
    "SEQ_ORGAO_PAI AS codpai, " + 
    "TIP_ORGAO AS sigla_tipoj, " + 
    "DSC_TIP_ORGAO AS tipo_oj, " + 
    "DSC_CIDADE AS cidade, " + 
    "SIG_UF AS uf, " + 
    "COD_IBGE AS codibge, " + 
    "TIP_ESFERA_JUSTICA AS esfera " + 
    "FROM serventias "
)

if carregar_dimensoes :
    df_qry_serventias.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
        .option("dbtable", "inovacnj.orgao_julgador") \
        .save()

print("tabela inovacnj.orgao_julgador criada.")

Py4JJavaError: An error occurred while calling o21584.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1310.0 failed 1 times, most recent failure: Lost task 1.0 in stage 1310.0 (TID 278588, f4bbee4c98b6, executor driver): org.postgresql.util.PSQLException: This connection has been closed.
	at org.postgresql.jdbc.PgConnection.checkClosed(PgConnection.java:885)
	at org.postgresql.jdbc.PgConnection.rollback(PgConnection.java:892)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:726)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:858)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:856)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:994)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.scala:994)
	at org.apache.spark.SparkContext.$anonfun$runJob$5(SparkCo

In [12]:
df_qry_classes = df_qry_classes.withColumnRenamed("descricao", "descclasse")
df_qry_movimentos = df_qry_movimentos.withColumnRenamed("descricao", "descmovimento")

In [13]:
# faz o carregamento de todos os arquivos em um único DataFrame,
# geracao do CSV com os dados consolidados
# cria a tabela fato com os movimentos processuais

basedir = "./base"

dirs_ramos_justica = [join(basedir, f) for f in os.listdir(basedir) if isdir(join(basedir, f))]

is_first = True

for dir_ramo_just in dirs_ramos_justica:
    print("Iniciando carregamento do ramo de justica: " + dir_ramo_just)
    dirs_tribunais = [join(dir_ramo_just, f) for f in os.listdir(dir_ramo_just) if isdir(join(dir_ramo_just, f))]
    
    for dir_trib in dirs_tribunais:
        print("Iniciando carregamento do tribunal: " + dir_trib)
        
        arquivos = [join(dir_trib, f) for f in os.listdir(dir_trib) if isfile(join(dir_trib, f))]
        
        df_union_tribunal = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
        
        for arq in arquivos:
            if arq.endswith(".DS_Store") :
                continue
                
            print("Carregando dataframe do arquivo: " + arq)
            df = spark.read.schema(schema).json(arq)
            df_union_tribunal = df_union_tribunal.union(df)
        
        # Cria uma view temporaria para o dataframe
        df_union_tribunal.createOrReplaceTempView("proc_movimentos")
        
        # Query para formato em CSV
        df_query_distinct = spark.sql(
            "SELECT DISTINCT " + 
            "siglaTribunal AS codtribunal, " + 
            "grau, " +
            "millisinsercao, " +

            "dadosBasicos.classeProcessual AS codclasse, " +
            "dadosBasicos.codigoLocalidade AS codlocalidade, " +
            "dadosBasicos.competencia, " +
            "to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') AS dtajuizamento, "
            "dadosBasicos.dscSistema AS descsistema, " +
            "dadosBasicos.nivelSigilo AS nivelsigilo, " +
            "dadosBasicos.numero AS npu, " + 
            "dadosBasicos.orgaoJulgador.codigoMunicipioIBGE AS oj_codibge, " +
            "dadosBasicos.orgaoJulgador.codigoOrgao AS oj_cod, " +
            "dadosBasicos.orgaoJulgador.instancia AS oj_instancia, " +
            "dadosBasicos.orgaoJulgador.nomeOrgao AS oj_descricao, " +
            "dadosBasicos.procEl AS tramitacao, " +
            "dadosBasicos.tamanhoProcesso AS tamanhoprocesso, " +
            "dadosBasicos.valorCausa AS valorcausa, " +

            "exp_assunto.assunto.codigoNacional AS ass_cod, " +
            "exp_assunto.assunto.principal AS ass_principal, " + 
            "exp_assunto.assunto.assuntoLocal.codigoAssunto AS ass_codlocal, " +
            "exp_assunto.assunto.assuntoLocal.codigoPaiNacional AS ass_codpainacional, " +
            "exp_assunto.assunto.assuntoLocal.descricao AS ass_desclocal, " +
            
            "exp_movimento.movimento.dataHora AS mov_dtmov, " +
            #"to_timestamp(exp_movimento.movimento.dataHora, 'yyyyMMddHHmmss') AS mov_dtmov, " +
            "exp_movimento.movimento.movimentoLocal.codigoMovimento AS mov_codlocal, " +
            "exp_movimento.movimento.movimentoLocal.codigoPaiNacional AS mov_codpainacional, " +
            "exp_movimento.movimento.movimentoNacional.codigoNacional AS mov_cod, " +
            "exp_movimento.movimento.nivelSigilo AS mov_nivelsigilo, " +

            "exp_movimento.movimento.orgaoJulgador.codigoMunicipioIBGE as mov_oj_codibge, " +
            "exp_movimento.movimento.orgaoJulgador.codigoOrgao as mov_oj_cod, " +
            "exp_movimento.movimento.orgaoJulgador.instancia as mov_oj_instancia, " +
            "exp_movimento.movimento.orgaoJulgador.nomeOrgao as mov_oj_descricao, " +

            "exp_movimento.movimento.tipoDecisao as mov_tpdecisao, " +
            "exp_movimento.movimento.tipoResponsavelMovimento as mov_tprespmov " +

            "FROM proc_movimentos " + 
            "LATERAL VIEW explode(dadosBasicos.assunto) exp_assunto as assunto " +
            "LATERAL VIEW explode(movimento) exp_movimento as movimento " + 
            "WHERE cast(substring(dadosBasicos.dataAjuizamento,0,4) as INT) >= 2000 AND to_timestamp(dadosBasicos.dataAjuizamento, 'yyyyMMddHHmmss') >= to_timestamp('20000101000000', 'yyyyMMddHHmmss') " + 
            "AND exp_movimento.movimento.movimentoNacional.codigoNacional NOT IN(581, 85, 12270, 12271) " + 
            "AND size(proc_movimentos.movimento) > 0 "
            "AND (proc_movimentos.movimento[0].movimentoNacional.codigoNacional IN (26, 12474) " +
            "AND proc_movimentos.movimento[size(proc_movimentos.movimento) -1].movimentoNacional.codigoNacional IN (22, 246)) "
        )
        
        df_movimentos_join = df_query_distinct \
           .join(df_qry_movimentos, df_query_distinct["mov_cod"] == df_qry_movimentos["cod"], "left") \
           .join(df_qry_classes, df_query_distinct["codclasse"] == df_qry_classes["cod"], "left") \
           .join(df_qry_assuntos, df_query_distinct["ass_cod"] == df_qry_assuntos["cod"], "left") \
           .select( \
                col("codtribunal"), col("grau"), col("millisinsercao"), col("codclasse"), col("descclasse"), \
                col("codlocalidade"), col("competencia"), col("dtajuizamento"), col("descsistema"), \
                col("nivelsigilo"), col("npu"), col("oj_codibge"), col("oj_cod"), \
                col("oj_instancia"), col("oj_descricao"), col("tramitacao"), col("tamanhoprocesso"), \
                col("valorcausa"), col("ass_cod"), col("descricao").alias("descassunto"), col("ass_principal"), col("ass_codlocal"), \
                col("ass_codpainacional"), col("ass_desclocal"), col("mov_dtmov"), col("mov_codlocal"), \
                col("mov_codpainacional"), col("mov_cod"), col("descmovimento"), col("mov_nivelsigilo"), col("mov_oj_codibge"), \
                col("mov_oj_cod"), col("mov_oj_instancia"), col("mov_oj_descricao"), col("mov_tpdecisao"), \
                col("mov_tprespmov"), col("natureza"), col("fase") \
        )
        
        df_query_distinctPd = df_movimentos_join.toPandas()
        df_query_distinctPd.to_csv('./output/movimentos_tribunais.csv', mode='a', header=is_first, sep = ";", index=False, chunksize=1000)
        
        df_movimentos_join = df_movimentos_join.withColumn('mov_dtmov', to_timestamp(df_movimentos_join['mov_dtmov'], 'yyyyMMddHHmmss'))
        
        if is_first == True:
            is_first = False
            df_movimentos_join.repartition(5).write \
                .mode("overwrite") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.fat_movimentos_te") \
                .option("batchsize", "10000") \
                .save()
        else :
            df_movimentos_join.repartition(5).write \
                .mode("append") \
                .format("jdbc") \
                .option("url", db_url).option("user", db_user).option("password", db_pass).option("driver", db_driver) \
                .option("dbtable", "inovacnj.fat_movimentos_te") \
                .option("batchsize", "10000") \
                .save()

        print("Finalizando carregamento do tribunal: " + dir_trib)
        
    print("Finalizando carregamento do ramo de justica: " + dir_ramo_just)
    
print("Carregamento dos arquivos finalizado.")


Iniciando carregamento do ramo de justica: ./base/justica_militar
Iniciando carregamento do tribunal: ./base/justica_militar/processos-tjmmg
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmmg/processos-tjmmg_2.json
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmmg/processos-tjmmg_3.json
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmmg/processos-tjmmg_1.json
Finalizando carregamento do tribunal: ./base/justica_militar/processos-tjmmg
Iniciando carregamento do tribunal: ./base/justica_militar/processos-tjmsp
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmsp/processos-tjmsp_2.json
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmsp/processos-tjmsp_3.json
Finalizando carregamento do tribunal: ./base/justica_militar/processos-tjmsp
Iniciando carregamento do tribunal: ./base/justica_militar/processos-tjmrs
Carregando dataframe do arquivo: ./base/justica_militar/processos-tjmrs/processo

Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt1/processos-trt1_5.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt1/processos-trt1_6.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt1/processos-trt1_1.json
Finalizando carregamento do tribunal: ./base/justica_trabalho/processos-trt1
Iniciando carregamento do tribunal: ./base/justica_trabalho/processos-trt8
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_1.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_6.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_5.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_4.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_3.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt8/processos-trt8_2.json
Fi

Finalizando carregamento do tribunal: ./base/justica_trabalho/processos-trt3
Iniciando carregamento do tribunal: ./base/justica_trabalho/processos-trt2
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_6.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_1.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_3.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_2.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_5.json
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt2/processos-trt2_4.json
Finalizando carregamento do tribunal: ./base/justica_trabalho/processos-trt2
Iniciando carregamento do tribunal: ./base/justica_trabalho/processos-trt5
Carregando dataframe do arquivo: ./base/justica_trabalho/processos-trt5/processos-trt5_5.json
Carregando dataframe do arquivo: ./bas

Finalizando carregamento do tribunal: ./base/justica_eleitoral/processos-tre-mg
Iniciando carregamento do tribunal: ./base/justica_eleitoral/processos-tre-al
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-al/processos-tre-al_1.json
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-al/processos-tre-al_2.json
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-al/processos-tre-al_3.json
Finalizando carregamento do tribunal: ./base/justica_eleitoral/processos-tre-al
Iniciando carregamento do tribunal: ./base/justica_eleitoral/processos-tre-mt
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-mt/processos-tre-mt_3.json
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-mt/processos-tre-mt_2.json
Carregando dataframe do arquivo: ./base/justica_eleitoral/processos-tre-mt/processos-tre-mt_1.json
Finalizando carregamento do tribunal: ./base/justica_eleitoral/processos-tre-mt
Iniciando 

Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_5.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_9.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_8.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_12.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_4.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_3.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_2.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_1.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_11.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjpr/processos-tjpr_7.json
Carregando dataframe do arquivo: ./base/justica_estadual/p

Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_5.json
Finalizando carregamento do tribunal: ./base/justica_estadual/processos-tjap
Iniciando carregamento do tribunal: ./base/justica_estadual/processos-tjal
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_1.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_11.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_7.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_6.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_10.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_9.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_5.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjal/processos-tjal_4.json


Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_7.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_6.json
Finalizando carregamento do tribunal: ./base/justica_estadual/processos-tjam
Iniciando carregamento do tribunal: ./base/justica_estadual/processos-tjac
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_4.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_12.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_8.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_9.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_5.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_2.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjac/processos-tjac_3.json
C

Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_8.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_4.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_5.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_9.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_6.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_7.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_12.json
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_1.json
Finalizando carregamento do tribunal: ./base/justica_estadual/processos-tjsp
Iniciando carregamento do tribunal: ./base/justica_estadual/processos-tjrs
Carregando dataframe do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_6.json
C

Finalizando carregamento do tribunal: ./base/justica_federal/processos-trf5
Iniciando carregamento do tribunal: ./base/justica_federal/processos-trf2
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_3.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_2.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_5.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_12.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_9.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_8.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_4.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_10.json
Carregando dataframe do arquivo: ./base/justica_federal/processos-trf2/processos-trf2_7.json
Carregando 